# Plotting tutorial

Some user input is required to be given in terms of astropy quantities (`astropy.unit.Quantity` objects). As such, we begin by importing astropy units.
Some matplotlib functions are also useful, but not nessecary.

In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt

## Loading a test data

The cosmoglobe plotting tools can be invoked on many different types of data
sets. Some of the functions are generalized to take arrays, cosmoglobe sky model
objects or strings of fits files.

In [ ]:
import cosmoglobe
chain = cosmoglobe.get_test_chain()
model = cosmoglobe.model_from_chain(chain, nside=256, components=["synch", "ff", "ame", "dust", "radio"])
dust = model.components["dust"].amp # Fetch the dust amplitude from the model object


In [ ]:
cosmoglobe.plot(dust)

A handy feature with the cosmoglobe map plotter is that it auto-formats specific components.
If the component is specified to be "dust", it will apply a suitible range, colormap and label.

In [ ]:
cosmoglobe.plot(dust, comp="dust")

If the input data has polarization data in it, the stokes parameter can also be specified to the autoformatter


In [ ]:
import healpy as hp
import cmasher 
cosmoglobe.plot(dust[2].value, min=-100, max=100, norm="symlog2", cmap="iceburn")
hp.projview(dust[2].value, min=-1e3, max=1e3, norm="symlog2",show_tickmarkers=True, cmap=cmasher.iceburn)

From here, you can go crazy with the plotting options, and any specified values will overwrite the autoformatting.

In [ ]:
cosmoglobe.plot(dust, sig="Q", ticks=[0, 3.14, None], norm="symlog2", unit=u.uK, fwhm=14*u.arcmin, nside=512, cmap="chroma", llabel="Left", rlabel="Right", title="Cool figure", width=7, graticule=True, projection_type="hammer", graticule_color="white", xtick_label_color="white", ytick_label_color="white", darkmode=True)
plt.gca().set_facecolor('black')

As already mentioned, the cosmoglobe plotter can take several different input types.
Simply pass it the path to your fits file, and it will plot it.

When using the cosmoglobe model object, there are even more features!
For example, one can look at the sky at a specific frequency

In [ ]:
cosmoglobe.plot(model, freq=70*u.GHz)

All these functions can be invoked with the command line as well, using "click" functionality.
This can be done by typing 
`cosmoglobe plot map.fits`

## Traceplot

Cosmoglobe comes with more than just sky-map plotting features.
It also allows the user to plot a quantity in a chain over Gibbs samples in a formatted way.

In [ ]:
#cosmoglobe.trace(chain, figsize=(10,2), sig=0, labels=["Reg1","Reg2","Reg3","Reg4"], dataset="synch/beta_pixreg_val", ylabel=r"$\beta_s^T$")
import numpy as np
data = np.random.rand(50, 2)
cosmoglobe.trace(data, labels=["Test 1","Test 2",], )

## Histogram

Histograms of the same values are also useful and can be accessed through

In [ ]:
cosmoglobe.hist(dust[1].value, figsize=(6,4), range=(-100,100), bins=40, prior=(-30,10))

## Gnomonic view

Cosmoglobe also provides gnomonic view plotting

In [ ]:
cosmoglobe.gnom(dust, comp="dust", norm="linear", sub=(2,2,1), fontsize={"title": 15,})
cosmoglobe.gnom(dust, comp="dust", norm="linear", sub=(2,2,2), fontsize={"title": 15,})
cosmoglobe.gnom(dust, comp="dust", norm="linear", sub=(2,2,3), fontsize={"title": 15,})
cosmoglobe.gnom(dust, comp="dust", norm="linear", sub=(2,2,4), fontsize={"title": 15,})

plt.tight_layout(pad=0.0,)

## Frequency spectrum

Another key feature of cosmoglobe is its ability to plot the full frequency spectrum of a given model.

In [ ]:
# Full page frequency spectrum in intensity and polarization
cosmoglobe.spec(model,)
cosmoglobe.spec(model,pol=True)
# Smaller version
cosmoglobe.spec(model, fraction=0.5, extend=False, comp_labelsize=8, band_labelsize=7, )

## Standalone colorbar

When writing papers it is often useful to have standalone colorbars. 
Cosmoglobe has this feature, just provide it with a colormap, ticks and a unit.

In [ ]:
cosmoglobe.standalone_colorbar("planck", ticks=[-0.2,0,0.2], unit=r"$S/\sigma_S$", )